<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/LangChain_OpenAI_Function_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain openai --upgrade --quiet

# OpenAI重大更新

## 2023-06-13 函数调用和其他API更新

[Function calling and other API updates](https://openai.com/blog/function-calling-and-other-api-updates)

一些令人兴奋的更新：

- 在Chat Completions API中新增了函数调用功能。
- 更新了更可控的gpt-4和gpt-3.5-turbo版本。
- 新增了gpt-3.5-turbo的16k上下文版本（相对于标准的4k版本）。
- 我们的最先进的嵌入模型成本降低了75%。
- gpt-3.5-turbo的输入令牌成本降低了25%。
- 公布了gpt-3.5-turbo-0301和gpt-4-0314模型的停用时间表。

**函数调用**功能使我们能够利用模型的自然语言理解能力，将人类语言有效地转化为结构化数据或在我们的代码中进行特定的函数调用。

### 基于OpenAI的Python SDK使用函数调用

In [2]:

import os,sys
import openai
from dotenv import load_dotenv, find_dotenv
# sys.path.append("../..")

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
print(find_dotenv())
_ = load_dotenv(find_dotenv())
print(os.environ["OPENAI_API_KEY"])
openai.api_key = os.environ["OPENAI_API_KEY"]
openai.api_base = os.environ["OPENAI_API_BASE"]

from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)


d:\Anaconda3\envs\LLM\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\Anaconda3\envs\LLM\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
d:\Anaconda3\envs\LLM\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


c:\Users\lenovo\Desktop\LangChainPlayGround\DeeperTutorials\.env
sk-lANo2jIeCWQt94UCCf5d16B7C32744279bF98b06C822D519


In [3]:
function_descriptions = [
  {
      "name": "get_student_score",
      "description": "Get the student score by given his or her name",
      "parameters": {
          "type": "object",
          "properties": {
              "name": {
                  "type": "string",
                  "description": "The student's name",
              }
          },
          "required": ["name"],
      },
    }
]

In [8]:

# messages = [
#     # SystemMessage(content="you are an unhelpful assistent for human, whenever a human say hi to you, you should say \" i Am JusT fine!!!!"),
#     # HumanMessage(content="Hi, I am a human, how are you today?"),
#     HumanMessage(content = """
#                  the alice scored 100, the bob scored 20, lucy's score is the average of alice and bob.
#                  What's the performance of Lucy in the school this year?
#                  """),
# ]

# user_query = ""

#     functions=function_descriptions,
#     function_call="auto",
# )
user_query = " the alice scored 100, the bob scored 20, lucy's score is the average of alice and bob.What's the performance of Lucy in the school this year?"

response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = [{"role": "user", "content": user_query}],
    functions=function_descriptions,
    function_call="auto",
)


# chat_llm = ChatOpenAI()
# chat_llm.bind(function_call="auto",functions=function_descriptions,)

In [12]:
ai_response_message = response["choices"][0]["message"]
print(ai_response_message)

print(response)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_student_score",
    "arguments": "{\n  \"name\": \"Lucy\"\n}"
  }
}
{
  "id": "chatcmpl-7o6NNYZ2U8pSZQcwqHTGBsHSBQWFz",
  "object": "chat.completion",
  "created": 1692175265,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_student_score",
          "arguments": "{\n  \"name\": \"Lucy\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 92,
    "completion_tokens": 17,
    "total_tokens": 109
  }
}


In [10]:
name = eval(ai_response_message['function_call']['arguments']).get("name")

In [11]:
name

'Lucy'

In [13]:
import json

SCORES = { 'Alex': 90, 'Lucy': 60 }
def get_student_score(name):

    """Get the student score by given his or her name"""

    score = {
        "name": name,
        "score": SCORES[name]
    }
    return json.dumps(score)

In [14]:
function_response = get_student_score(name=name)

function_response

'{"name": "Lucy", "score": 60}'

In [17]:
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": user_query},
        ai_response_message,
        {
            "role": "function",
            "name": "get_student_score",
            "content": function_response,
        },
    ],
)
print(second_response)

{
  "id": "chatcmpl-7o6Pwy1sEvAKxf486CA3XGwysNPfp",
  "object": "chat.completion",
  "created": 1692175424,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The performance of Lucy in the school this year is 60."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 81,
    "completion_tokens": 13,
    "total_tokens": 94
  }
}


In [18]:
print (second_response['choices'][0]['message']['content'])

The performance of Lucy in the school this year is 60.


### 基于LangChain框架使用函数调用

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage
from langchain.tools import format_tool_to_openai_function, YouTubeSearchTool

In [21]:
llm = ChatOpenAI()

#### LangChain工具(Tool)的OpenAI函数调用能力

In [22]:
tools = [YouTubeSearchTool()]
functions = [format_tool_to_openai_function(t) for t in tools]

In [23]:
functions

[{'name': 'youtube_search',
  'description': 'search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional',
  'parameters': {'properties': {'__arg1': {'title': '__arg1',
     'type': 'string'}},
   'required': ['__arg1'],
   'type': 'object'}}]

In [24]:
message = llm.predict_messages([HumanMessage(content='search videos in the topic of OpenAI on Youtube')], functions=functions)

In [25]:
message.additional_kwargs

{'function_call': {'name': 'youtube_search',
  'arguments': '{\n  "arg1": "OpenAI"\n}'}}

#### LangChain使用OpenAI函数调用实例

In [27]:
function_descriptions = [
    {
        "name": "remove_word_from_string",
        "description": "Remove a word from a string by given its index",
        "parameters": {
            "type": "object",
            "properties": {
                "string": {
                    "type": "string",
                    "description": "The original string to be processed",
                },
                "index": {
                    "type": "integer",
                    "description": "The index of the word to be removed"
                },
            },
            "required": [
                "string",
                "index"
            ],
        },
    },
    {
        "name": "send_message_by_email",
        "description": "Send an email with the text message to a recipient",
        "parameters": {
            "type": "object",
            "properties": {
                "recipient": {
                    "type": "string",
                    "description": "The email address of the recipient",
                },
                "message": {
                    "type": "string",
                    "description": "The message of the email content",
                }
            },
            "required": [
                "recipient",
                "message"
            ],
        },
    }
]


In [28]:
question = """
I have a string as follows:

black yellow red blue green

Please do the following 2 operations on it:
1. Remove the third word in the string
2. Send the updated string to Alex via email alex@xyz.com
"""

Helper functions to get function parameter names. They will be used to implement dynamic function calls in Python.

In [29]:
import inspect

def get_function_parameter_names(function):
  if function is not None and inspect.isfunction(function):
      parameter_names = inspect.signature(function).parameters.keys()
      return list(parameter_names)
  else:
      return None

In [30]:
def remove_word_from_string(string, index):
    words = string.split()

    if 0 <= index < len(words):
        del words[index]

        return ' '.join(words)
    else:
        return string

def send_message_by_email(recipient, message):
    print(f'Sending {message} to {recipient}')
    return f'Just sent email to {recipient}'

In [31]:
parameter_names = get_function_parameter_names(remove_word_from_string)
parameter_names

['string', 'index']

In [32]:
first_response = llm.predict_messages([HumanMessage(content=question)], functions=function_descriptions)
first_response

AIMessage(content='', additional_kwargs={'function_call': {'name': 'remove_word_from_string', 'arguments': '{\n"string": "black yellow red blue green",\n"index": 2\n}'}}, example=False)

In [33]:
first_response.additional_kwargs

{'function_call': {'name': 'remove_word_from_string',
  'arguments': '{\n"string": "black yellow red blue green",\n"index": 2\n}'}}

In [36]:
# Get function name, and its arguments

function_name = first_response.additional_kwargs["function_call"]["name"]
arguments = json.loads(first_response.additional_kwargs["function_call"]["arguments"])

# Locate the function and make the call
the_function = globals().get(function_name)
parameter_names = get_function_parameter_names(the_function)
parameter_values = []
for parameter_name in parameter_names:
  parameter_values.append(arguments[parameter_name])

returned_value = the_function(*parameter_values)

In [37]:
print(returned_value)

black yellow blue green


In [38]:
second_response = llm.predict_messages(
    [
        HumanMessage(content=question),
        AIMessage(content=str(first_response.additional_kwargs)),
        ChatMessage(
            role='function',
            additional_kwargs = {'name': function_name},
            content = returned_value
        )
    ],
    functions=function_descriptions
)

In [39]:
second_response.additional_kwargs

{'function_call': {'name': 'send_message_by_email',
  'arguments': '{\n  "recipient": "alex@xyz.com",\n  "message": "black yellow blue green"\n}'}}

In [40]:
second_response.content

'The updated string after removing the third word is "black yellow blue green". \n\nNow I will send the updated string to Alex via email.'

In [41]:
# Again get function name, and its arguments

function_name = second_response.additional_kwargs["function_call"]["name"]
arguments = json.loads(second_response.additional_kwargs["function_call"]["arguments"])

# Locate the function and make the call
the_function = globals().get(function_name)
parameter_names = get_function_parameter_names(the_function)
parameter_values = []
for parameter_name in parameter_names:
  parameter_values.append(arguments[parameter_name])

returned_value = the_function(*parameter_values)

Sending black yellow blue green to alex@xyz.com


In [42]:
# Print the returned value in the second function call
print(returned_value)

Just sent email to alex@xyz.com


In [43]:
third_response = llm.predict_messages(
    [
        HumanMessage(content=question),
        AIMessage(content=str(first_response.additional_kwargs)),
        AIMessage(content=str(second_response.additional_kwargs)),
        ChatMessage(
            role='function',
            additional_kwargs = {'name': function_name},
            content = returned_value
        )
    ], functions=function_descriptions
)

In [44]:
third_response

AIMessage(content='I have removed the third word from the string and sent the updated string "black yellow blue green" to Alex via email at alex@xyz.com.', additional_kwargs={}, example=False)